In [ ]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from glob import glob
import segmentation_models_pytorch as smp

# 1. 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv_file, input_dir, gt_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.input_dir = input_dir
        self.gt_dir = gt_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_path = os.path.join(self.input_dir, self.data.iloc[idx]['input_image_path'])
        gt_path = os.path.join(self.gt_dir, self.data.iloc[idx]['gt_image_path'])

        # 이미지 로드
        input_image = Image.open(input_path).convert('L')  # 흑백
        gt_image = Image.open(gt_path).convert('RGB')  # 컬러

        if self.transform:
            input_image = self.transform(input_image)
            gt_image = self.transform(gt_image)

        return input_image, gt_image

# 2. 데이터 전처리 정의
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # [-1, 1] 정규화
])

# 3. 데이터셋 및 데이터로더 생성
train_dataset = ImageDataset(
    csv_file='./train.csv',
    input_dir='',
    gt_dir='',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# 4. 모델 정의
class FullPipeline(nn.Module):
    def __init__(self):
        super().__init__()
        self.mask_model = smp.Unet(
            encoder_name="resnet34",        
            encoder_weights="imagenet",     
            in_channels=1,                  
            classes=1                       
        )
        self.inpaint_model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=2,  # 이미지 + 마스크
            classes=1
        )
        self.colorize_model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=1,
            classes=3
        )

    def forward(self, x):
        # Step 1: Mask Detection
        mask = self.mask_model(x)
        
        # Step 2: Inpainting
        input_with_mask = torch.cat([x, mask], dim=1)  # 채널 합치기
        inpainted = self.inpaint_model(input_with_mask)
        
        # Step 3: Colorization
        colorized = self.colorize_model(inpainted)
        
        return mask, inpainted, colorized

# 5. 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FullPipeline().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.L1Loss()

# 6. 학습 루프
from tqdm import tqdm

for epoch in range(10):
    model.train()
    total_loss = 0
    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # 모델 예측
        mask, inpainted, colorized = model(inputs)

        # 손실 계산
        loss = criterion(inpainted, inputs) + criterion(colorized, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1:   0%|          | 0/500 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import os


def test_model(model, test_csv, input_dir, transform, save_dir):
    # 테스트 데이터셋 생성
    test_data = pd.read_csv(test_csv)
    input_paths = test_data["input_image_path"]

    model.eval()  # 모델 평가 모드
    os.makedirs(save_dir, exist_ok=True)  # 결과 저장 디렉토리 생성

    with torch.no_grad():
        for idx, input_path in enumerate(input_paths):
            # 입력 이미지 로드 및 전처리
            full_input_path = os.path.join(input_dir, input_path)
            input_image = Image.open(full_input_path).convert("L")  # 흑백
            input_tensor = (
                transform(input_image).unsqueeze(0).to(device)
            )  # 배치 차원 추가

            # 모델 예측
            _, _, colorized = model(input_tensor)

            # 결과를 numpy 형식으로 변환
            output_np = colorized[0].cpu().permute(1, 2, 0).numpy()  # 컬러화된 결과

            # 이미지 범위 복구 (Normalize 후 값 범위 복원)
            output_np = (
                (output_np * 255).clip(0, 255).astype("uint8")
            )  # 0-255로 스케일링

            # 파일명 설정 (TEST_000, TEST_001, ...)
            save_path = os.path.join(save_dir, f"TEST_{idx:03d}.png")
            Image.fromarray(output_np).save(save_path)

            print(f"Result saved to: {save_path}")


# 8. 테스트 실행
test_csv = "./test.csv"  # 테스트 CSV 파일 경로
input_dir = "./test_inputs"  # 테스트 입력 이미지 디렉토리
save_dir = "./sample_submission"  # 결과 저장 디렉토리

test_model(model, test_csv, input_dir, transform, save_dir)